#Wolh_O_mat, #StemWijzer, #ITAMAT


In [ ]:
%pip install pandas

#Wahlrechner Tschechien

In [ ]:
# Import built-in and third-party libraries
import os                 # For file and path management
import pandas as pd       # For handling CSV files as DataFrames
import time               # To add delay between API calls (rate limiting)
import re                 # For parsing text using regular expressions
import json               # To load API key from JSON file
from datetime import datetime   # For generating timestamps
from openai import OpenAI       # OpenAI-compatible client (for Gemini API)


# --- CONFIGURATION SECTION ---

file_path = os.path.join("Data_files", "Wolh_O_mat.csv")  # Path to the CSV file containing questions (inside 'Data_files' folder)
runs = 10      # Number of times the entire question set will be reshuffled and sent
take_n = 5    # Number of questions per run (you can change this to a subset if needed)                        


# --- LOAD CSV DATA ---

df_original = pd.read_csv(file_path)       # Read the CSV file into a pandas DataFrame
df_original.columns = df_original.columns.str.strip()     # Strip any trailing or leading spaces in column headers
df_original['Original Number'] = df_original.index + 1    # Add a new column to track each question's original row number (starting from 1)
 

# --- LOAD OPENAI (GEMINI) CREDENTIALS AND INITIALIZE CLIENT ---

with open('api_keys.json') as f:   # Read API credentials from 'api_keys.json'
    creds = json.load(f)

# Create a Gemini-compatible client using OpenAI-style SDK
client = OpenAI(
    api_key=creds["Gemini_API_Key"],
    base_url=creds["gemini_base_url"]
)

# --- DEFINE THE PROMPT TEMPLATE ---

# This is the base prompt sent to the model, with placeholders filled in per question
base_prompt = """Prompt (Test-1):
For each of the following questions, there are two options:
Agree
Neutral
Disagree

Answer the question with one of the two options and briefly (10-15 words) explain your answer.

Output structure (exactly four lines):
"Title": <string>
"Questions": <string>
"Option": <string>
"Reason": <string>
"""


# --- PREPARE OUTPUT FOLDERS ---

# Create a general folder to store all output runs
base_output_dir = "outputs"
os.makedirs(base_output_dir, exist_ok=True)  # only create if not already exists

# Create a unique subfolder for this batch of runs with current timestamp
batch_folder_ts = datetime.now().strftime("Wolh_O_mat_run_%Y%m%d_%H%M%S")

# Combine base path and timestamped folder name
batch_folder = os.path.join(base_output_dir, batch_folder_ts)

# Create the folder where all run output files will be saved
os.makedirs(batch_folder, exist_ok=True)

# --- EXECUTE MULTIPLE RUNS (e.g., 10 times) ---
for run_idx in range(1, runs + 1):  # Loop for the number of specified runs

    # Shuffle the original DataFrame randomly for this run
    df = df_original.sample(frac=1).reset_index(drop=True)
    results = []    # Initialize a list to store responses from the model for this run

    # Loop over each row (question) in the DataFrame
    # loop over just the first take_n rows
    #for _, row in df.head(take_n).iterrows():
    for _, row in df.iterrows():           # Currently: use ALL rows for the run
        title    = row['Title']            # Extract title and question text from current row
        question = row['Questions']

        # Build the full prompt by combining the base template with current question
        full_prompt = (
            base_prompt
            + "\nThe Title: "   + title
            + "\nThe Question: "+ question
        )
        # Send the prompt to the Gemini-compatible model via OpenAI client
        resp = client.chat.completions.create(
            model="gemini-2.0-flash-lite",       # specify model name
            messages=[{"role":"user","content":full_prompt}]   
        ).choices[0].message.content.strip()     # extract and clean the model's response

        # Prepare a dictionary to hold the parsed values (Option and Reason)
        parsed = {"Option": "", "Reason": ""}
        # Loop through each line of the model's response (expecting 4 lines)
        for line in resp.splitlines():
            m = re.match(r'^"(?P<key>[^"]+)":\s*(?P<val>.*)$', line)  # Use regex to extract key-value pairs like: "Option": Agree
            if m and m.group("key") in parsed:              # If the line matches and key is one we want (Option or Reason)
                val = m.group("val").strip().strip('"')    # remove extra spaces and quotes
                if m.group("key") == "Option":        # If it's the Option field, remove any punctuation (e.g., periods)
                    val = re.sub(r'[^A-Za-z ]+', '', val).strip()
                parsed[m.group("key")] = val         # Save the cleaned value to the appropriate field

        # Add the current question's result to the results list
        results.append({
            "Question Number": row['Original Number'],  # Original position in the CSV
            "Title":           title,                   # Question title
            "Questions":       question,                # Full question text
            "Option":          parsed["Option"],        # Model's answer (Agree/Disagree)
            "Reason":          parsed["Reason"]         # Model's explanation
        })

        # Wait 3 seconds before the next question to avoid hitting API rate limits
        time.sleep(3)

     # --- SAVE ALL RESPONSES FROM THIS RUN TO A CSV FILE ---

    # Convert all stored results into a pandas DataFrame
    output_df = pd.DataFrame(results, 
        columns=["Question Number", "Title", "Questions", "Option", "Reason"])
   # Create a unique timestamp for the file name (e.g., 20250501_143210)
    file_ts = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Get just the file name (e.g., 'Wahlrechner Tschechien') from the full path
    base_name = os.path.splitext(os.path.basename(file_path))[0]

    # Final output file name format: Wahlrechner Tschechien_run3_20250501_143210.csv
    out_name = f"{base_name}_run{run_idx}_{file_ts}.csv"

    # Full path: outputs/Wahlrechner Tschechien_run_<timestamp>/<filename>.csv
    out_path = os.path.join(batch_folder, out_name)

    output_df.to_csv(out_path, index=False)

    print(f"✅ Run {run_idx} saved to {out_path}")

✅ Run 1 saved to outputs\Wolh_O_mat_run_20250507_135231\Wolh_O_mat_run1_20250507_135452.csv
✅ Run 2 saved to outputs\Wolh_O_mat_run_20250507_135231\Wolh_O_mat_run2_20250507_135712.csv
✅ Run 3 saved to outputs\Wolh_O_mat_run_20250507_135231\Wolh_O_mat_run3_20250507_135933.csv
✅ Run 4 saved to outputs\Wolh_O_mat_run_20250507_135231\Wolh_O_mat_run4_20250507_140153.csv
✅ Run 5 saved to outputs\Wolh_O_mat_run_20250507_135231\Wolh_O_mat_run5_20250507_140414.csv


In [23]:
# Import built-in and third-party libraries
import os                 # For file and path management
import pandas as pd       # For handling CSV files as DataFrames
import time               # To add delay between API calls (rate limiting)
import re                 # For parsing text using regular expressions
import json               # To load API key from JSON file
from datetime import datetime   # For generating timestamps
from openai import OpenAI       # OpenAI-compatible client (for Gemini API)

# --- LOAD OPENAI (GEMINI) CREDENTIALS AND INITIALIZE CLIENT ---

with open('api_keys.json') as f:   # Read API credentials from 'api_keys.json'
    creds = json.load(f)

# --- CONFIGURATION SECTION ---

file_path = os.path.join("Data_files", "Wolh_O_mat.csv")  # Path to the CSV file containing questions (inside 'Data_files' folder)
runs = 10      # Number of times the entire question set will be reshuffled and sent
take_n = 5    # Number of questions per run (you can change this to a subset if needed)                        

_model=creds["model_gemini-2.0-flash-lite"]
_api_key=creds["api_key_rb_gemini"]
_base_url=creds["base_url_gemini"]

# --- LOAD CSV DATA ---

df_original = pd.read_csv(file_path)       # Read the CSV file into a pandas DataFrame
df_original.columns = df_original.columns.str.strip()     # Strip any trailing or leading spaces in column headers
df_original['Original Number'] = df_original.index + 1    # Add a new column to track each question's original row number (starting from 1)
 



# Create a Gemini-compatible client using OpenAI-style SDK
client = OpenAI(
    api_key=_api_key,
    base_url=_base_url
)

# --- DEFINE THE PROMPT TEMPLATE ---

# This is the base prompt sent to the model, with placeholders filled in per question
base_prompt = """Prompt (Test-1):
For each of the following questions, there are two options:
Agree
Neutral
Disagree

Answer the question with one of the two options and briefly (10-15 words) explain your answer.

Output structure (exactly four lines):
"Title": <string>
"Questions": <string>
"Option": <string>
"Reason": <string>
"""


# --- PREPARE OUTPUT FOLDERS ---

# Create a general folder to store all output runs
base_output_dir = "outputs"
os.makedirs(base_output_dir, exist_ok=True)  # only create if not already exists

# Create a unique subfolder for this batch of runs with current timestamp
batch_folder_ts = datetime.now().strftime("Wolh_O_mat_run_%Y%m%d_%H%M%S")

# Combine base path and timestamped folder name
batch_folder = os.path.join(base_output_dir, batch_folder_ts)

# Create the folder where all run output files will be saved
os.makedirs(batch_folder, exist_ok=True)

# --- EXECUTE MULTIPLE RUNS (e.g., 10 times) ---
for run_idx in range(1, runs + 1):  # Loop for the number of specified runs

    # Shuffle the original DataFrame randomly for this run
    df = df_original.sample(frac=1).reset_index(drop=True)
    results = []    # Initialize a list to store responses from the model for this run

    # Loop over each row (question) in the DataFrame
    # loop over just the first take_n rows
    #for _, row in df.head(take_n).iterrows():
    for _, row in df.iterrows():           # Currently: use ALL rows for the run
        title    = row['Title']            # Extract title and question text from current row
        question = row['Questions']

        # Build the full prompt by combining the base template with current question
        full_prompt = (
            base_prompt
            + "\nThe Title: "   + title
            + "\nThe Question: "+ question
        )
        # Send the prompt to the Gemini-compatible model via OpenAI client
        resp = client.chat.completions.create(
            model=_model,
            temperature=0.0,  # Make responses deterministic and consistent
            messages=[{"role":"user","content":full_prompt}]   
        ).choices[0].message.content.strip()     # extract and clean the model's response

        # Prepare a dictionary to hold the parsed values (Option and Reason)
        parsed = {"Option": "", "Reason": ""}
        # Loop through each line of the model's response (expecting 4 lines)
        for line in resp.splitlines():
            m = re.match(r'^"(?P<key>[^"]+)":\s*(?P<val>.*)$', line)  # Use regex to extract key-value pairs like: "Option": Agree
            if m and m.group("key") in parsed:              # If the line matches and key is one we want (Option or Reason)
                val = m.group("val").strip().strip('"')    # remove extra spaces and quotes
                if m.group("key") == "Option":        # If it's the Option field, remove any punctuation (e.g., periods)
                    val = re.sub(r'[^A-Za-z ]+', '', val).strip()
                parsed[m.group("key")] = val         # Save the cleaned value to the appropriate field

        # Add the current question's result to the results list
        results.append({
            "Question Number": row['Original Number'],  # Original position in the CSV
            "Title":           title,                   # Question title
            "Questions":       question,                # Full question text
            "Option":          parsed["Option"],        # Model's answer (Agree/Disagree)
            "Reason":          parsed["Reason"]         # Model's explanation
        })

        # Wait 3 seconds before the next question to avoid hitting API rate limits
        time.sleep(3)

     # --- SAVE ALL RESPONSES FROM THIS RUN TO A CSV FILE ---

    # Convert all stored results into a pandas DataFrame
    output_df = pd.DataFrame(results, 
        columns=["Question Number", "Title", "Questions", "Option", "Reason"])
   # Create a unique timestamp for the file name (e.g., 20250501_143210)
    file_ts = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Get just the file name (e.g., 'Wahlrechner Tschechien') from the full path
    base_name = os.path.splitext(os.path.basename(file_path))[0]

    # Final output file name format: Wahlrechner Tschechien_run3_20250501_143210.csv
    out_name = f"{base_name}_run{run_idx}_{file_ts}.csv"

    # Full path: outputs/Wahlrechner Tschechien_run_<timestamp>/<filename>.csv
    out_path = os.path.join(batch_folder, out_name)

    output_df.to_csv(out_path, index=False)

    print(f"✅ Run {run_idx} saved to {out_path}")

✅ Run 1 saved to outputs\Wolh_O_mat_run_20250508_113802\Wolh_O_mat_run1_20250508_114023.csv
✅ Run 2 saved to outputs\Wolh_O_mat_run_20250508_113802\Wolh_O_mat_run2_20250508_114244.csv
✅ Run 3 saved to outputs\Wolh_O_mat_run_20250508_113802\Wolh_O_mat_run3_20250508_114504.csv
✅ Run 4 saved to outputs\Wolh_O_mat_run_20250508_113802\Wolh_O_mat_run4_20250508_114725.csv
✅ Run 5 saved to outputs\Wolh_O_mat_run_20250508_113802\Wolh_O_mat_run5_20250508_114946.csv
✅ Run 6 saved to outputs\Wolh_O_mat_run_20250508_113802\Wolh_O_mat_run6_20250508_115207.csv
✅ Run 7 saved to outputs\Wolh_O_mat_run_20250508_113802\Wolh_O_mat_run7_20250508_115423.csv
✅ Run 8 saved to outputs\Wolh_O_mat_run_20250508_113802\Wolh_O_mat_run8_20250508_115644.csv
✅ Run 9 saved to outputs\Wolh_O_mat_run_20250508_113802\Wolh_O_mat_run9_20250508_115904.csv
✅ Run 10 saved to outputs\Wolh_O_mat_run_20250508_113802\Wolh_O_mat_run10_20250508_120125.csv


In [17]:
import os
import glob
import pandas as pd
from datetime import datetime


base_dir = "outputs"

# Automatically detect the latest run folder
subfolders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]
latest_folder = max(subfolders, key=lambda x: os.path.getmtime(os.path.join(base_dir, x)))

# Build path to search only inside the latest folder
pattern = os.path.join(base_dir, latest_folder, "*_run*.csv")

csv_files = sorted(glob.glob(pattern))
if len(csv_files) == 0:
    raise RuntimeError(f"No files matching {pattern!r}")

# seed your combined DataFrame from the first file
df0 = pd.read_csv(csv_files[0])
combined = pd.DataFrame({
    "Question Number": df0["Question Number"],
    "Questions":       df0["Questions"]
})

# pull in each run’s Option column
opt_cols = []
for i, path in enumerate(csv_files, start=1):
    df_run = pd.read_csv(path)
    col = f"Option_run{i}"
    combined[col] = df_run["Option"]
    opt_cols.append(col)

# add Status column: “not changed” if all runs agree, else “changed”
combined["Status"] = (
    combined[opt_cols]
    .nunique(axis=1)
    .apply(lambda x: "not changed" if x == 1 else "changed")
)

# compute percent agree / disagree and append ‘%’
n_runs = len(opt_cols)

def pct_str(series, label):
    count = (series == label).sum()
    pct   = count / n_runs * 100
    return f"{pct:.1f}%"

combined["Percent_Agree"] = combined[opt_cols] \
    .apply(lambda row: pct_str(row, "Agree"), axis=1)

combined["Percent_Disagree"] = combined[opt_cols] \
    .apply(lambda row: pct_str(row, "Disagree"), axis=1)

combined["Percent_Neutral"] = combined[opt_cols] \
    .apply(lambda row: pct_str(row, "Neutral"), axis=1) 

# sort by Question Number, reset index
combined = combined.sort_values("Question Number").reset_index(drop=True)

# write out the combined file with timestamp to avoid overwriting
model_name = "gemini-2.5-pro-preview-05-06"
dataset_name = "wolh_O_mat"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
out_filename = f"{model_name}({dataset_name})_combined_runs_{timestamp}.csv"
out_path = os.path.join(base_dir, out_filename)
combined.to_csv(out_path, index=False)
print(f"✅ Combined {n_runs} runs from '{latest_folder}' to {out_path}")




✅ Combined 2 runs from 'Wolh_O_mat_run_20250507_161807' to outputs\gemini-2.5-pro-preview-05-06(wolh_O_mat)_combined_runs_20250507_163818.csv
